In [4]:
# Paquetes
import os
import pandas as pd
import re
import string
import numpy as np

# Para eliminar palabrotas
import spanlp
#from spanlp.palabrota import Palabrota


In [5]:
# Ruta de archivos a procesar
path_original = '/home/daniel/Documents/GitHub/AIMemeGenerator'
#path_original = os.getcwd()
path = path_original + '/data/scrapper'
os.chdir(path)
os.getcwd()
# ruta de directorio a crear
#path_bases_no_tildes = path_original + "/data/bases_no_tildes"
#os.makedirs(path_bases_no_tildes,0o007)


'/home/daniel/Documents/GitHub/AIMemeGenerator/data/scrapper'

In [6]:
files = os.listdir()
files = [val for val in files if re.search(r'.csv\Z', val)]

In [7]:
files = [#['scrapymemes10m.csv',
#'scrapymemes33m.csv',
#'scrapymemes9m.csv',
#'scrapymemes11m.csv',
#'scrapymemes12m.csv',
#'scrapymemes22m.csv',
#'scrapymemes2mp2.csv',
#'scrapymemes17m.csv',
#'scrapymemes18m.csv',
#'scrapymemes25m.csv',
#'scrapymemes13m.csv',
#'scrapymemes21m.csv',
#'scrapymemes26m.csv',
#'scrapymemes7m.csv',
#'scrapymemes20m.csv',
#'scrapymemes19m.csv',
'scrapymemes2mp2.csv']

In [8]:
files

['scrapymemes2mp2.csv']

In [9]:
# Parámetros: Campos para reemplazar tildes
x = "aaaaaaeeeeiiiioooooouuuuy"       # Tildes
y = "àáâãäåèéêëìíîïðòóôõöùúûüý"       # Caradteres que no fueron identificados por utf-8
char_replace = {'Ã“':"O", 'Ã‘':'O','Ã­':'I',
                'Ã‰':'E','Ãš':'U','â™':'u','âˆ':'e','ÃŒ':'I',
                'Ãˆ':'E','Ãš':'U','â€':'A','Ã':'A','Ã±':'n','Ã‘':'ñ'}
#files=  files[0]

exceptions = ["ano","pendejo","pendeja","estupido","estupida","joder","tonto","tonta","pete","marinero","bobo",
             "perro","idiotas","idiota","computadora","joda","dios","jodas","jode","la computadora",
              "imbecil","pagan","paja","paga","ojo","internet","rata","baboso","bochinche","boletoso",
             "bombril","cachon","guiso","hoyo","jartera","lampara","lamparoso","lichigo","mamera","orto",
             "tontarron","cojones","globalizacion"]

include_words = ["verga","culaso","maricon","marico","putear","putona","puti","putita"]


# groserias
from spanlp.palabrota import Palabrota
from spanlp.domain.strategies import CosineSimilarity
from spanlp.domain.strategies import JaccardIndex

# Limpieza de texto
from spanlp.domain.strategies import Preprocessing, TextToLower, NumbersToVowelsInLowerCase ,NumbersToConsonantsInLowerCase, ExpandAbbreviations
from spanlp.domain.countries import Country

#strategies = [TextToLower(), NumbersToVowelsInLowerCase(), 
#             NumbersToConsonantsInLowerCase(), ExpandAbbreviations()] # Defino mis estrategias de limpieza o pre-procesamiento

#cosine = CosineSimilarity(0.95, normalize=False) 
#jaccard = JaccardIndex(threshold=0.95, normalize=False, n_gram=2)
palabrota = Palabrota(countries=[Country.COLOMBIA,Country.ESPANA, Country.ARGENTINA], exclude=exceptions, 
                       include=include_words) #,
#                     distance_metric=cosine)


# Objetos
sin_convertir = {}     # Guardará índices de filas que no se pudieron convertir o de groserías.
word_list     = []     # Listado de caracteres
rows_to_drop  = [' Personalizado','plantilla']
list_print    = np.arange(0,1000000,10000)

for file in files:
    print("Archivo:",file)
    file_name = re.sub('\.csv$', '', file) 
    # Cargar archivos:
    base = pd.read_csv("./"+file,encoding="utf-8")                            # Cargar archivo
    base.drop_duplicates(subset=['plantilla','texto'],inplace=True,ignore_index=True) # Eliminar duplicados
    base = base[~base.plantilla.isin(rows_to_drop)]
    base["texto_no_tilde"] = ""                                           # Crear campo de texto para almacenar tildes
    base["palabrota"]     = ""
    id_no_encode = []                                                     # Lista para almacenar code_id     

    print("Cantidad de registros:",base.shape[0])
    print(" ")
    index = 0            # índice en cero para iterar por filas
    # Loop para convertir caracteres de latin-1 a utf-8 y guardar información en nueva tabla
    for t in base.texto:
    #for index in range(0,base.shape[0]):
        if index in list_print:
            print(index)
        try:
            #t = base.values[index,2]
            #t = base.iat[index,2]
            tab_map = t.maketrans(y, x)
            t = t.translate(tab_map).strip()
            t =  t.encode("latin-1").decode("utf-8")
            for a,b in char_replace.items():
                t = t.replace(a,b)
            t = t.translate(tab_map).strip()
            #base.values[index,3] = t
            base.iat[index,3] = t
            # Filtro de groserías
            base.iat[index,4] = palabrota.contains_palabrota(t)
            #base.values[index,4] = palabrota.contains_palabrota(t)
            index += 1
        # Si no logra convertir un caracter, guardar el índice dentro de la base y continuar.    
        except UnicodeEncodeError:
            id_no_encode.append(index)
            index += 1
       #     array = t.split() 
       #     for k in array:
       #         try:
       #             k.encode("latin-1").decode("utf-8")
       #         except:
       #             word_list.append(k)
       #         pass
       #     pass
        except UnicodeDecodeError:
            id_no_encode.append(index)
            index += 1
       #     array = t.split() 
       #     for k in array:
       #         try:
       #             k.encode("latin-1").decode("utf-8")
       #         except:
       #             word_list.append(k)
       #         pass
       #     pass        
    # Guardar archivos limpios
    base = base[base["texto_no_tilde"] != ""]
    #base = base[base["palabrota"]==False]
    base = base[base.palabrota == 0]
    str_file = path_original + "/data/bases_no_groserias/" + file_name + "_no_tilde.csv"
    base.to_csv(str_file, index=False )
    del base, index, id_no_encode
    # Guardar índices
  #  sin_convertir[file_name] = id_no_encode
   # print("registros sin limpieza:",len(id_no_encode))
    # print("Expresiones sin limpiar:",len(word_list))
    print(" ")
# word_list = list(dict.fromkeys(word_list))

Archivo: scrapymemes2mp2.csv
Cantidad de registros: 205502
 
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
 


In [22]:
os.listdir(r'/home/daniel/Documents/GitHub/AIMemeGenerator/data/bases_no_groserias')

['scrapymemes33m_no_tilde.csv',
 'scrapymemes7m_no_tilde.csv',
 'scrapymemes16m_no_tilde.csv',
 'scrapymemes32m_no_tilde.csv',
 'scrapymemes20m_no_tilde.csv',
 'scrapymemes6m_no_tilde.csv',
 'scrapymemes1m_no_tilde.csv',
 'scrapymemes24m_no_tilde.csv',
 'scrapymemes10m_no_tilde.csv',
 'scrapymemes9m_no_tilde.csv',
 'scrapymemes5m_no_tilde.csv',
 'scrapymemes8m_no_tilde.csv',
 'scrapymemes28m_no_tilde.csv',
 'scrapymemes21m_no_tilde.csv',
 'scrapymemes19m_no_tilde.csv',
 'scrapymemes15m_no_tilde.csv',
 'scrapymemes17m_no_tilde.csv',
 'scrapymemes22m_no_tilde.csv',
 'scrapymemes11m_no_tilde.csv',
 'scrapymemes18m_no_tilde.csv',
 'scrapymemes23m_no_tilde.csv',
 'scrapymemes25m_no_tilde.csv',
 'scrapymemes31m_no_tilde.csv',
 'scrapymemes13m_no_tilde.csv',
 'scrapymemes30m_no_tilde.csv',
 'scrapymemes29m_no_tilde.csv',
 'scrapymemes14m_no_tilde.csv',
 'scrapymemes12m_no_tilde.csv',
 'scrapymemes4m_no_tilde.csv',
 'scrapymemes3m_no_tilde.csv',
 'scrapymemes26m_no_tilde.csv',
 'scrapymemes27m